In [1]:
!pip install pymorphy2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 21.5 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=95537f6f857abe4b4c54c6889e28ecb28cd5f15e4590b5a8a3b73ba0239eb808
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [2]:
POS_TAGS = [
    "adj",
    "adv",
    "intj",
    "noun",
    "propn",
    "verb",
    "adp",
    "aux",
    "cconj",
    "det",
    "num",
    "part",
    "pron",
    "sconj",
    "x"
]

In [18]:
import csv
import pymorphy2

morph = pymorphy2.MorphAnalyzer()


class SearchEngineCSV:
    def __init__(self, csv_file):
        self.sentences = []
        self.tokens = []
        self.lemmas = []
        self.pos_tags = []
        self.chapters = []

        with open(csv_file, newline='', encoding='utf-8') as file:
            reader = csv.reader(file)
            next(reader)
            for row in reader:
                self.sentences.append(row[0])
                self.tokens.append([token.lower() for token in row[1].split(';')])  # Токены в нижнем регистре
                self.lemmas.append([lemma.lower() for lemma in row[2].split(';')])  # Леммы в нижнем регистре
                self.pos_tags.append(row[3].split(';'))
                self.chapters.append(row[4])

    def match_token(self, token, word, lemma, pos):
        token = token.lower()  # Приведение токена к нижнему регистру
        if '"' in token:
            return token.strip('"') == word

        if '+' in token:
            query_word, query_pos = token.split('+')
            return lemma.lower() == query_word.lower() and pos.lower() == query_pos.lower()

        if token in POS_TAGS:
            return pos.lower() == token

        return lemma == morph.parse(token)[0].normal_form

    def match_sequence(self, query_tokens, words, lemmas, pos_tags):
        if len(query_tokens)>1 and '"' in query_tokens[2]:
            query_tokens[1] = query_tokens[1] + '"'
        for j, query_token in enumerate(query_tokens):
            query_token = query_token.lower()  # Приведение токена запроса к нижнему регистру
            if query_token in POS_TAGS:
                if pos_tags[j].lower() != query_token:
                    return False
            else:
                if not self.match_token(query_token, words[j], lemmas[j], pos_tags[j]):
                    return False
        return True

    def search(self, query):
        query_tokens = query.lower().split()  # Приведение запроса к нижнему регистру

        matches = []
        for idx, sentence in enumerate(self.sentences):
            words = self.tokens[idx]
            lemmas = self.lemmas[idx]
            pos_tags = self.pos_tags[idx]
            if len(words) != len(lemmas) or len(words) != len(pos_tags):  # проверка, что количество токенов, лемм и тегов частей речи одинаковое
                continue

            # Поиск последовательности токенов в предложении
            for i in range(len(words) - len(query_tokens) + 1):
                formatted_sentence = ""
                matches_in_sent = []
                # Проверка последовательности для n-граммы
                if self.match_sequence(query_tokens, words[i:i+len(query_tokens)], lemmas[i:i+len(query_tokens)], pos_tags[i:i+len(query_tokens)]):
                    # Форматируем предложение с источником
                    source_info = f"[{self.chapters[idx]}]"
                    if formatted_sentence == "":
                        formatted_sentence = f"{sentence} {source_info}"
                    matches_in_sent.append((i, i + len(query_tokens)))  # информация о том, какие слова - ответ на запрос (номер первого слова последовательности и первого слова после неё)
                if formatted_sentence != "":
                    matches.append([formatted_sentence, matches_in_sent])  # сначала строка "текст предложения [номер главы]", потом список мест, в которых слова, отвечающие на запрос

        return matches


# Пример использования
csv_file = '/content/corpus_data_with_labels.csv'  # Путь к файлу
search_engine = SearchEngineCSV(csv_file)

# Код для запроса и вывода результатов
print('Введите запрос:')
for_search = input()
results = search_engine.search(for_search)
print(f'Количество примеров: {len(results)}')
set_results = {}
for result in results:
    print(result)

Введите запрос:
дерево+NOUN
Количество примеров: 1
['Множество низеньких домиков, разбросанных по берегу Терека, который разбегается шире и шире, мелькали из-за дерев, а дальше синелись зубчатою стеной горы, и из-за них выглядывал Казбек в своей белой кардинальской шапке. [Герой нашего времени. Часть 1. Максим Максимыч. - rvb.ru - https://rvb.ru/19vek/lermontov/lp/text/01-03.html]', [(14, 15)]]


In [20]:
# Примеры запросов
print(search_engine.search('сказать'))  # Любая форма слова "сказать"
print(search_engine.search('"меня"'))  # Точная форма "меня"
print(search_engine.search('человек+noun'))  # Лемма "человек" с POS-тегом "S"
print(search_engine.search('NOUN ADV VERB'))  # Существительное, глагол, наречие в последовательности

[['Вы мне опять скажете, что человек не может быть так дурен, а я вам скажу, что ежели вы верили возможности существования всех трагических и романтических злодеев, отчего же вы не веруете в действительность Печорина? [Герой нашего времени. Предисловие. - rvb.ru - https://rvb.ru/19vek/lermontov/lp/text/01-01.html]', [(3, 4)]], ['Вы мне опять скажете, что человек не может быть так дурен, а я вам скажу, что ежели вы верили возможности существования всех трагических и романтических злодеев, отчего же вы не веруете в действительность Печорина? [Герой нашего времени. Предисловие. - rvb.ru - https://rvb.ru/19vek/lermontov/lp/text/01-01.html]', [(14, 15)]], ['Вы скажете, что нравственность от этого не выигрывает? [Герой нашего времени. Предисловие. - rvb.ru - https://rvb.ru/19vek/lermontov/lp/text/01-01.html]', [(1, 2)]], ['— Скажите, пожалуйста, отчего это вашу тяжелую тележку четыре быка тащат шутя, а мою пустую шесть скотов едва подвигают с помощию этих осетин? [Герой нашего времени. Часть